Instance Generator 

In [1]:
import tkinter as tk
from tkinter import messagebox
import random
import math

class InstanceGeneratorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Instance Generator")

        # Define the bold font
        bold_font = ("Helvetica", 10, "bold")

        # Center the input fields and labels
        self.frame = tk.Frame(root)
        self.frame.pack(pady=20)

        self.label_tracks = tk.Label(self.frame, text="Number of Tracks (T):", font=bold_font)
        self.label_tracks.grid(row=0, column=0, padx=10, pady=10)
        self.entry_tracks = tk.Entry(self.frame)
        self.entry_tracks.grid(row=0, column=1, padx=10, pady=10)

        self.label_slots = tk.Label(self.frame, text="Slots per Track (S):", font=bold_font)
        self.label_slots.grid(row=1, column=0, padx=10, pady=10)
        self.entry_slots = tk.Entry(self.frame)
        self.entry_slots.grid(row=1, column=1, padx=10, pady=10)

        self.button_calculate = tk.Button(self.frame, text="Calculate", command=self.calculate_slots, bg="dark blue", fg="white")
        self.button_calculate.grid(row=2, column=0, columnspan=2, padx=10, pady=10)
    
    def calculate_slots(self):
        try:
            self.T = int(self.entry_tracks.get())
            self.S = int(self.entry_slots.get())
            total_slots = self.T * self.S

            self.label_total_slots = tk.Label(self.frame, text=f"Total Slots: {total_slots}")
            self.label_total_slots.grid(row=3, column=0, columnspan=2, padx=10, pady=10)

            bold_font = ("Helvetica", 10, "bold")
            self.label_occupancy = tk.Label(self.frame, text="Occupancy Percentage (%):", font=bold_font)
            self.label_occupancy.grid(row=4, column=0, padx=10, pady=10)
            self.entry_occupancy = tk.Entry(self.frame)
            self.entry_occupancy.grid(row=4, column=1, padx=10, pady=10)

            self.label_instances = tk.Label(self.frame, text="Number of Instances:", font=bold_font)
            self.label_instances.grid(row=5, column=0, padx=10, pady=10)
            self.entry_instances = tk.Entry(self.frame)
            self.entry_instances.grid(row=5, column=1, padx=10, pady=10)

            self.button_generate = tk.Button(self.frame, text="Generate Instances", command=self.generate_instances, bg="dark blue", fg="white")
            self.button_generate.grid(row=6, column=0, columnspan=2, padx=10, pady=10)

            self.button_export = tk.Button(self.frame, text="Export Instances", command=self.export_instances, bg="dark blue", fg="white")
            self.button_export.grid(row=7, column=0, columnspan=2, padx=10, pady=10)

        except ValueError:
            messagebox.showerror("Invalid input", "Please enter valid integers for T and S.")

    def generate_instances(self):
        try:
            self.occupancy_percentage = float(self.entry_occupancy.get())
            number_of_instances = int(self.entry_instances.get())
            total_slots = self.T * self.S
            allotted_slots = math.floor(total_slots * (self.occupancy_percentage / 100))

            block_ids = list(range(allotted_slots))
            self.instances = []

            for _ in range(number_of_instances):
                random.shuffle(block_ids)
                instance = [['' for _ in range(self.S)] for _ in range(self.T)]

                block_index = 0
                track_order = list(range(self.T))
                random.shuffle(track_order)

                for track in track_order:
                    for slot in range(self.S-1, -1, -1):
                        if block_index < allotted_slots:
                            instance[track][slot] = block_ids[block_index]
                            block_index += 1

                self.instances.append(instance)

            self.display_instances(self.instances)

        except ValueError:
            messagebox.showerror("Invalid input", "Please enter valid values for occupancy percentage and number of instances.")

    def display_instances(self, instances):
        instance_window = tk.Toplevel(self.root)
        instance_window.title("Generated Instances")

        canvas = tk.Canvas(instance_window)
        scrollbar = tk.Scrollbar(instance_window, orient="vertical", command=canvas.yview)
        scrollable_frame = tk.Frame(canvas)

        scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(
                scrollregion=canvas.bbox("all")
            )
        )

        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)

        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")

        occupancy_label = int(self.occupancy_percentage)

        for i, instance in enumerate(instances):
            label_instance = tk.Label(scrollable_frame, text=f"Label: FYMP_T{self.T}_S{self.S}_O{occupancy_label}_{i}")
            label_instance.grid(row=i * (self.T + 3), column=0, columnspan=self.S+1, padx=10, pady=10)

            for track in range(self.T):
                for slot in range(self.S):
                    slot_content = instance[self.T - track - 1][slot] if instance[self.T - track - 1][slot] != '' else '-'
                    label_slot = tk.Label(scrollable_frame, text=slot_content, borderwidth=1, relief="solid", width=5, height=2)
                    label_slot.grid(row=i * (self.T + 3) + track + 1, column=slot, padx=2, pady=2)

                track_name = f"Track {self.T - track}"
                label_track = tk.Label(scrollable_frame, text=track_name)
                label_track.grid(row=i * (self.T + 3) + track + 1, column=self.S, padx=10, pady=2)

            slot_names = "   ".join(f"Slot {slot + 1}" for slot in range(self.S))
            label_slots = tk.Label(scrollable_frame, text=slot_names)
            label_slots.grid(row=i * (self.T + 3) + self.T + 1, column=0, columnspan=self.S+1, padx=10, pady=10)

    def export_instances(self):
        occupancy_label = int(self.occupancy_percentage)
        with open("FYMP.txt", "w") as file:
            for i, instance in enumerate(self.instances):
                file.write(f"Label: FYMP_T{self.T}_S{self.S}_O{occupancy_label}_{i}\n")
                for track in range(len(instance)):
                    track_name = f"Track {self.T - track}"
                    track_content = "   ".join(
                        f"{instance[self.T - track - 1][slot]}" if instance[self.T - track - 1][slot] != '' else '-'
                        for slot in range(self.S)
                    )
                    file.write(f"{track_content}   {track_name}\n")
                file.write("\n")
        messagebox.showinfo("Export Complete", "Instances exported to Instances.txt")


if __name__ == "__main__":
    root = tk.Tk()
    app = InstanceGeneratorApp(root)
    root.mainloop()